In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper-parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [10]:
# MNIST datasets
train_dataset = torchvision.datasets.MNIST(root='../../data/', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                          download= True)
test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size,
                                         shuffle=False)

# Fully connected neural network with one hidden layer
"""
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet,self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size,num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)
"""
class neuralnet(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(neuralnet,self).__init__()
        self.layer1 = nn.Linear(input_size,hidden_size)
        self.layer2 = nn.Linear(hidden_size,hidden_size)
        self.layer3 = nn.Linear(hidden_size,output_size)
        
    def forward(self,x):
        x= nn.functional.relu(self.layer1(x))
        x = self.layer2(x)
        x = nn.functional.relu(x)
        x= self.layer3(x)
        return x 

model = neuralnet(input_size,hidden_size,num_classes,).to(device)

In [11]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs,labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch: [1/5], Step: [100/600], Loss: 0.2924
Epoch: [1/5], Step: [200/600], Loss: 0.2321
Epoch: [1/5], Step: [300/600], Loss: 0.2816
Epoch: [1/5], Step: [400/600], Loss: 0.1541
Epoch: [1/5], Step: [500/600], Loss: 0.1678
Epoch: [1/5], Step: [600/600], Loss: 0.1241
Epoch: [2/5], Step: [100/600], Loss: 0.1548
Epoch: [2/5], Step: [200/600], Loss: 0.0571
Epoch: [2/5], Step: [300/600], Loss: 0.0825
Epoch: [2/5], Step: [400/600], Loss: 0.2032
Epoch: [2/5], Step: [500/600], Loss: 0.0326
Epoch: [2/5], Step: [600/600], Loss: 0.1197
Epoch: [3/5], Step: [100/600], Loss: 0.0505
Epoch: [3/5], Step: [200/600], Loss: 0.0459
Epoch: [3/5], Step: [300/600], Loss: 0.0935
Epoch: [3/5], Step: [400/600], Loss: 0.0997
Epoch: [3/5], Step: [500/600], Loss: 0.0627
Epoch: [3/5], Step: [600/600], Loss: 0.0496
Epoch: [4/5], Step: [100/600], Loss: 0.0407
Epoch: [4/5], Step: [200/600], Loss: 0.0628
Epoch: [4/5], Step: [300/600], Loss: 0.0336
Epoch: [4/5], Step: [400/600], Loss: 0.0257
Epoch: [4/5], Step: [500/600], L

In [7]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()
        
    print('Accuracy of the network on the 10000 test images: {} %'
         .format(100 * correct / total))

Accuracy of the network on the 10000 test images: 97.9 %


In [8]:
torch.save(model.state_dict(), './checkpoint/perceptron')

FileNotFoundError: [Errno 2] No such file or directory: './checkpoint/perceptron'